# Notebooks sur Kaggle
Kaggle nous offre la possibilité d'utiliser Jupyter Notebook.
Les données d'entrainement et de test sont disponible en lecture seule.

In [18]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

/kaggle/input


In [ ]:
!pip install timm


# Remarques
- C'est à vous de diviser l'ensemble d'entrainement en train/valid, vous pouvez vous en inspirez de la question 1
- Ce code est à titre indicatif, vous n'êtes pas obligés de suivre la même logique pour lader les données

In [ ]:
# nous utilisons pathlib afin de parcourir les dossiers et fichiers
import pathlib

# nous utilisons aussi torchvision et pytorch pour le code de Data Loading
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split

from PIL import Image # pour lire les images .jpg

# nous utilisons Pandas pour manipuler et creer le fichier de soumission assez rapidement
import pandas as pd

import numpy as np
from shutil import copyfile

import timm
import torch
import torch.nn as nn


In [2]:
train_path = '/kaggle/input/glo-7030-ou-suis-je-h2024/train'
valid_path = '/content/drive/MyDrive/TP2_deepL/valid' # a changer selon le chemin dans votre machine

# le chemin vers le test set
test_path = '/kaggle/input/glo-7030-ou-suis-je-h2024/test'

ModuleNotFoundError: No module named 'google.colab'

In [22]:
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

"""
Cette fonction sépare les images de CUB200 en un jeu d'entraînement et de test.

dataset_path: Path où se trouve les images de CUB200
train_path: path où sauvegarder le jeu d'entraînement
test_path: path où sauvegarder le jeu de test
"""
def separate_train_valid(dataset_path, train_path, test_path):

    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        if classname.startswith('.'):
            continue
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            if file.startswith('.'):
                continue
            file_path = os.path.join(dataset_path, classname, file)
            if i < 1500:
                copyfile(file_path, os.path.join(test_path, classname, file))
            else:
                copyfile(file_path, os.path.join(train_path, classname, file))
            i += 1

        class_index += 1

In [23]:
separate_train_valid(dataset_path=train_path+"_init", train_path=train_path, test_path=test_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/TP2_deepL/train_init_init'

In [24]:
train_transform = transforms.Compose([
     transforms.Resize((72, 72)),
     transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
     transforms.Resize((72, 72)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [25]:
# utilisons la classe ImageFolder afin de charger le train set
train_dataset = datasets.ImageFolder(train_path, train_transform)

# la classe ImageFolder assigne automatiquement un label pour chaque nom de classe (class -> idx)
print('class -> idx : ',train_dataset.class_to_idx)

# on aura besoin d'un dictionnaire qui fait le sens inverse (idx -> class)
idx_to_class = {train_dataset.class_to_idx[class_name]: class_name for class_name in  train_dataset.class_to_idx}
print('idx -> class : ',idx_to_class)

class -> idx :  {'Boston': 0, 'London': 1, 'Montreal': 2, 'Paris': 3, 'Quebec': 4}

idx -> class :  {0: 'Boston', 1: 'London', 2: 'Montreal', 3: 'Paris', 4: 'Quebec'}


In [26]:
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size

train_dataset, valid_dataset = random_split(train_dataset, [train_size, test_size])


In [33]:
len(train_dataset), len(valid_dataset)

(14960, 3741)

Creation du modèle

In [27]:
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=5)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [29]:
from tqdm import tqdm

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        # Phase d'entraînement
        model.train()
        train_loss_items = []
        correct_train = 0
        total_train = 0
        train_tqdm = tqdm(train_loader, desc=f'Training Epoch {epoch+1}', leave=False)
        for images, labels in train_tqdm:
            images = images.to(device)  # Envoyer les images au GPU
            labels = labels.to(device)  # Envoyer les étiquettes au GPU

            optimizer.zero_grad()  # Effacer les gradients existants
            outputs = model(images)  # Passer les images à travers le modèle
            loss = criterion(outputs, labels)  # Calculer la perte
            loss.backward()  # Rétropropagation des erreurs
            optimizer.step()  # Ajuster les paramètres du modèle
            train_loss_items.append(loss.item())

            # Calculer l'accuracy courante
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            current_train_accuracy = 100 * correct_train / total_train

            train_tqdm.set_postfix(loss=loss.item(), accuracy=f'{current_train_accuracy:.2f}%')

        avg_train_loss = sum(train_loss_items) / len(train_loss_items)
        print(f'Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {current_train_accuracy:.2f}%')

        # Phase de validation
        model.eval()
        val_loss_items = []
        correct_val = 0
        total_val = 0
        val_tqdm = tqdm(val_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images, labels in val_tqdm:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss_items.append(loss.item())
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                current_val_accuracy = 100 * correct_val / total_val

                val_tqdm.set_postfix(loss=loss.item(), accuracy=f'{current_val_accuracy:.2f}%')

        avg_val_loss = sum(val_loss_items) / len(val_loss_items)
        print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {current_val_accuracy:.2f}%')


In [32]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
model.to(device)
train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Using cuda device


KeyboardInterrupt: 

## Chargement des données de Test
Pour les données de test, nous n'avons pas d'étitquettes, donc la classe ImageFolder n'est pas utile.
- Nous avons besoin des noms des images afin de créer le fichier de soumission

In [30]:
# on hérite de la classe Dataset de PyTorch
class TestDataset(Dataset):
    def __init__(self, test_path, transform=None):

        # lister toutes les images dans le repertoire test
        # self.image_paths va etre une liste contenant des
        # elements de type Path (pour plus d'info voir la class Path de pathlib)
        # cela fonctionne seulement pour les versions de python >3.4
        self.image_paths = list(pathlib.Path(test_path).glob('*.jpg'))

        # trier les noms des fichiers d'images par ordre
        self.image_paths.sort()

        # garder la fonctions de tranform dans self pour l'utiliser dans __getitem__
        self.transform = transform

    def __getitem__(self, index):
        # index est un nombre qui vient du dataloader (il est entre 0 et ce que retourne la methode __len__ ci-dessous)
        # c'est donc entre 0 et le nombre d'images de test
        img_path = self.image_paths[index]

        # retourner aussi le nom de l'image en question(i.e. '0xxxx')
        # sans l'extension '.jpg', donc on ignore les 4 dernier caracteres du nom de l'image (.jpg) avec [:-4]
        # par exemple, pour l'image '00037.jpg' on retourne '00037'
        # cela va vous etre util pour generer le fichier predictions.csv
        img_name = img_path.name[:-4] #img_path est un objet de type Path, et a donc un attribut 'name'

        # lire l'image avec PIL
        img = Image.open(img_path)

        # appliquer les transforms s'il y'en a
        if self.transform is not None:
            img = self.transform(img)

        return img, img_name

    def __len__(self):
        return len(self.image_paths)

In [ ]:
# Nous allons tester la classe TestDataset
test_dataset = TestDataset(test_path, transform=transforms.ToTensor()) # ici je transforme les images en Tensor pour une utilisation rapide de PyTorch Dataloader

# on aura besoin d'un dataloader qui enveloppe notre objet test_dataset
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4) # souvent le shuffle est mis a False, mais avec notre implementation, ca marche dans tous les cas

In [ ]:
# on itere sur le dataloader pour récuperer une batch
batch, names = next(iter(test_loader))

i = 6 # choisir une image dans la batch
print(names[i]) # afficher le nom de l'image
transforms.ToPILImage()(batch[i]) # afficher l'image correspondante à partir de la batch

In [ ]:
# affichons l'image a partir du dossier test et voir si c'est la meme
nom_image = names[i]+'.jpg' # rajouter l'extension
Image.open(test_path + '/' + nom_image)

# Generation du fichier de prédiction
Pour chaque batch que l'on passe au model, nous gardons dans une liste, les noms des images et les labels prédits   
Le résultats pourrait ressembler à ça  
image_names   `['00000', '00001', '00002', ..., '09453']`   
label_predictions   `[1, 0, 4, ..., 1]`


In [ ]:
# generons une prediction aleatoire pour notre test

label_predictions = []
image_names = []
for batch,im_names in test_loader:
    # remplir avec des predictions aleatoires (entre 0 et 4)
    random_preds = [np.random.randint(0, 5) for _ in range(len(batch))]
    label_predictions.extend(random_preds)

    # retenir les noms des images
    image_names.extend(im_names)

assert len(label_predictions) == len(image_names)
assert len(label_predictions) == len(test_dataset) # est-ce qu'on a prédits tous les exemples de tests ?
print(f'Il y a {len(label_predictions)} exemples de test')

In [ ]:
# Utilisons Pandas afin de generer un DataFrame

predictions_df = pd.DataFrame(data=zip(image_names, label_predictions), columns=['image_name', 'class_label'])
predictions_df

In [ ]:
# mais il faudra d'abord traduire les class_label en nom de classes
# on peut utiliser le dictionnaire idx_to_class calculé au départ
predictions_df['class'] = predictions_df['class_label'].map(idx_to_class)
predictions_df

In [ ]:
# on drop la colonne class_label avant d'entregistrer en fichier CSV
predictions_df = predictions_df.drop(labels=['class_label'], axis=1)

In [ ]:
# enregistrer dans fichier de format CSV
predictions_df.to_csv('./mes_predictions.csv', index=None)